# O1: Clasificacion del Fallecimiento

El objetivo de esta seccion en el proyecto es **"predecir si un paciente falleció a partir de las demas caracteristicas contenidas en el dataset"**. Esta tarea es clave para poder identificar de manera rapida, con un modelo de clasificacion, a que pacientes se les requiere un seguimiento o intervencion con prioridad

### 1. Configuracion Inicial para el desarrollo del proyecto

Para poder empezar a desarrollar el proyecto tenemos que configurar el entorno de trabajo instalando las bibliotecas necesarias requeridas:

- Pandas: para lecutar y manopulacion del dataset

- numpy: para operaciones numericas y manejo de arrays

- scikit-learn: para poder crear y evaluar modelos mediante: clasificacion, regresion, validacion cruzada, ajuste de hiperparámetros.

- matplotlib: para la visualizacion de los resultados y graficos



Primero instalamos las librerias mencionadas si no las tenemos instaladas

!pip install pandas numpy scikit-learn matplotlib

Posteriormente las importamos en nuestro proyecto para poder: cargar datos, transformarlos, dividirlos en conjuntos de entrenamiento y prueba y para luego modelarlos de forma sencilla

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer


### 2. Carga y Exploracion del Dataset

Cargamos el archivo del dataset que queremos trabajar (en formato csv) e imprimimos las primeras lineas del dataset y la informacion general del DataFrame

In [2]:
# Leer el archivo CSV
df = pd.read_csv("custom_covid19.csv")

# Mostrar las primeras filas para ver el contenido
print(df.head())

# Información general del DataFrame
print(df.info())


   USMER  MEDICAL_UNIT  SEX  PATIENT_TYPE   DATE_DIED  INTUBED  PNEUMONIA  \
0      2            12    1             1  9999-99-99       97          2   
1      2            12    2             1  9999-99-99       97          2   
2      2             4    2             1  9999-99-99       97          2   
3      2             9    1             1  9999-99-99       97          2   
4      1            12    2             1  9999-99-99       97          2   

   AGE  PREGNANT  DIABETES  ...  ASTHMA  INMSUPR  HYPERTENSION  OTHER_DISEASE  \
0   41         2         2  ...       2        2             2              2   
1   57         2         1  ...       2        2             2              2   
2   38         2         2  ...       2        2             2              2   
3   68         2         2  ...       2        2             2              2   
4   63         2         2  ...       2        2             1              2   

   CARDIOVASCULAR  OBESITY  RENAL_CHRONIC  TOBACCO

Podemos ver las principales lineas del dataset en las que cada columna tiene un rango de valor dependiendo de su tipo de variable.



## 3. Procesamiento y Limpieza de Datos

### 3.1 Crear la Variable "Fallecido"

La columna "DATE_DIED" tiene el valor por defecto de "9999-99-99" cuando el paciente no esta fallecido, por lo que vamos a transformarla en una nueva variable binaria de valor 1 - yes ; 2 - no

In [3]:
df['Fallecido'] = df['DATE_DIED'].apply(lambda x: 2 if x == '9999-99-99' else 1)


Volvemos a imprimir las primeras filas del dataset para poder comprobar que se han asignado los valores de la columna "fallecidoº" correctamente

In [4]:
# Mostrar las primeras filas para ver el contenido
print(df.head())

# Información general del DataFrame
print(df.info())

   USMER  MEDICAL_UNIT  SEX  PATIENT_TYPE   DATE_DIED  INTUBED  PNEUMONIA  \
0      2            12    1             1  9999-99-99       97          2   
1      2            12    2             1  9999-99-99       97          2   
2      2             4    2             1  9999-99-99       97          2   
3      2             9    1             1  9999-99-99       97          2   
4      1            12    2             1  9999-99-99       97          2   

   AGE  PREGNANT  DIABETES  ...  INMSUPR  HYPERTENSION  OTHER_DISEASE  \
0   41         2         2  ...        2             2              2   
1   57         2         1  ...        2             2              2   
2   38         2         2  ...        2             2              2   
3   68         2         2  ...        2             2              2   
4   63         2         2  ...        2             1              2   

   CARDIOVASCULAR  OBESITY  RENAL_CHRONIC  TOBACCO  TEST_RESULT  ICU  \
0               2        2

### 3.2 Conversion de Variables Booleanas

Existen variables en el dataset con valores de datos erroneos/perdidos por lo que tenemos que procesarlos de forma parecida a la nueva variable y establecer valores "no determinados".

Por lo que vamos a establecer a los valores distintos a los correspondientes (1- yes , 2 - no) el valor no determinado "np.nam" a dichas variables

In [5]:
colums = [
    'USMER', 'INTUBED', 'PNEUMONIA', 'PREGNANT', 'DIABETES', 
    'COPD', 'ASTHMA', 'INMSUPR', 'HYPERTENSION', 'OTHER_DISEASE', 
    'CARDIOVASCULAR', 'OBESITY', 'RENAL_CHRONIC', 'TOBACCO', 'ICU'
]

for col in colums:
    df[col] = df[col].replace({97: np.nan, 98: np.nan, 99: np.nan})


Mostramos los valores para ver si los cambios se han realizado correctamente

In [6]:
# Mostrar las primeras filas para ver el contenido
print(df.head())


   USMER  MEDICAL_UNIT  SEX  PATIENT_TYPE   DATE_DIED  INTUBED  PNEUMONIA  \
0      2            12    1             1  9999-99-99      NaN        2.0   
1      2            12    2             1  9999-99-99      NaN        2.0   
2      2             4    2             1  9999-99-99      NaN        2.0   
3      2             9    1             1  9999-99-99      NaN        2.0   
4      1            12    2             1  9999-99-99      NaN        2.0   

   AGE  PREGNANT  DIABETES  ...  INMSUPR  HYPERTENSION  OTHER_DISEASE  \
0   41       2.0       2.0  ...      2.0           2.0            2.0   
1   57       2.0       1.0  ...      2.0           2.0            2.0   
2   38       2.0       2.0  ...      2.0           2.0            2.0   
3   68       2.0       2.0  ...      2.0           2.0            2.0   
4   63       2.0       2.0  ...      2.0           1.0            2.0   

   CARDIOVASCULAR  OBESITY  RENAL_CHRONIC  TOBACCO  TEST_RESULT  ICU  \
0             2.0      2.0

## 4. Modelado con la variable "Fallecido"

Al haber generado el modelo de clasificacion del fallecido (O1), la variable objetivo es "Fallecido" con los valores binario (1- yes ; 2- no)

### 4.1 Preparacion para la clasificaicion

Extraemos 'X' e 'Y'


In [7]:
X = df.drop(columns=['DATE_DIED', 'Fallecido'])
y = df['Fallecido']

### 4.2 Division de los datos y creacion del modelo

Para crear el modelo, dividimos los datos y entrenamos el modelo.

In [8]:

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un imputer. En este ejemplo usaremos 'most_frequent' ya que 
# es adecuado para variables categóricas o datos discretos.
imputer = SimpleImputer(strategy='most_frequent')

# Ajustar el imputer en X_train y transformar tanto X_train como X_test
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Entrenar un modelo de regresión logística y ajustar el hiperparámetro C
param_grid = {'C': [0.1, 1, 10, 100]}
model_lr = LogisticRegression(max_iter=1000)
grid_lr = GridSearchCV(model_lr, param_grid, cv=5, scoring='accuracy')
grid_lr.fit(X_train_imputed, y_train)

# Predicciones y evaluación usando los datos imputados
y_pred = grid_lr.predict(X_test_imputed)
print("Mejores hiperparámetros:", grid_lr.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Mejores hiperparámetros: {'C': 10}
Accuracy: 0.94645
              precision    recall  f1-score   support

           1       0.70      0.51      0.59      1523
           2       0.96      0.98      0.97     18477

    accuracy                           0.95     20000
   macro avg       0.83      0.75      0.78     20000
weighted avg       0.94      0.95      0.94     20000



## 5. Evaluacion e Interpretacion del modelo de clasificacion obtenido

### 5.1 Revision de los resultados

Podemos observar que los pacientes que fallecieron (tipo 1) tiene una precision del 70% y un recall del 57%, no obstante, para los no fallecidos (tipo 2) los valores son muy altos con una precision del 96% y un recall del 98%.

Esto indica que el modelo es muy bueno para poder identificar los pacientes que no fallecieron pero le cuesta predecir correctamente la clase de los fallecidos.

Para poder ajustar este modelo, podems modificar el umbral de decision o cambiar de modelo de clasificacion (p.e: RandomForestClassifier) para intentar mejorar el recall (la sensibilidad) en el tipo 1 (fallecidos)



A continuacion, vamos a cargar los archivos CSV de validacion dado para nuestro proyecto, los cuales mostraran un conjunto de pruebas a las que debemos someter nuestro modelo


In [9]:
# Importa pandas si aún no lo has hecho
import pandas as pd

# Cargar el conjunto de prueba de características y las etiquetas verdaderas
proj_test_data = pd.read_csv("proj-test-data.csv")
proj_test_class = pd.read_csv("proj-test-class.csv")

# Visualiza las primeras filas para revisar su estructura
print(proj_test_data.head())
print(proj_test_class.head())

   USMER  MEDICAL_UNIT  SEX  PATIENT_TYPE   DATE_DIED  INTUBED  PNEUMONIA  \
0      1            12    2             1  9999-99-99       97          2   
1      2            12    2             1  9999-99-99       97          2   
2      2            12    1             1  9999-99-99       97          2   
3      1             4    2             1  9999-99-99       97          2   
4      2             4    1             1  9999-99-99       97          2   

   AGE  PREGNANT  DIABETES  ...  ASTHMA  INMSUPR  HYPERTENSION  OTHER_DISEASE  \
0   32         2         2  ...       2        2             2              2   
1   37         2         2  ...       2        2             2              2   
2   45         2         2  ...       2        2             2              2   
3   49         2         2  ...       2        2             2              2   
4   31         2         2  ...       2        2             2              2   

   CARDIOVASCULAR  OBESITY  RENAL_CHRONIC  TOBACCO

### 5.2 Preprocesamiento del conjunto de Prueba

Para hacer que el modelo funcione, el conjunto de prueba debe de ser transformado al mismo formato al cual hemos transformado nuestro dataset base por el cual se basa nuestro modelo.

Primero eliminamos la columna DATE_DIED:


In [10]:
X_proj_test = proj_test_data.drop(columns=['DATE_DIED'])

Despues, tenemos que usar el mismo objeto "imputer" que hemos entreando para el conjunto de entrenamiento:

In [11]:
imputer = SimpleImputer(strategy='most_frequent')
X_train_imputed = imputer.fit_transform(X_train)  

Utilizamos este imputer para transformar el conjunto de pruebas:


In [12]:
X_proj_test_imputed = imputer.transform(X_proj_test)

### 5.3 Evaluacion del modelo de clasificacion

Por ultimo, entrenamos el modelo(utilizando GridSearchCV; guardado en grid_lr) para predecir el resultado en los datos de prueba, el cual posteriormente compara las predicciones con las del archivo "proj-test-class.csv"

In [13]:

y_proj_pred = grid_lr.predict(X_proj_test_imputed)

print("Accuracy en el conjunto de prueba independiente:", accuracy_score(proj_test_class, y_proj_pred))
print(classification_report(proj_test_class, y_proj_pred))


Accuracy en el conjunto de prueba independiente: 0.04
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        93
           1       0.80      0.57      0.67         7
           2       0.00      0.00      0.00         0

    accuracy                           0.04       100
   macro avg       0.27      0.19      0.22       100
weighted avg       0.06      0.04      0.05       100



c:\Users\super\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\super\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\super\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

# O2 Predecir la edad del paciente a partir de las demas variables

El objetivo O2 consite en predecir la edad de los pacientes utilizando las demas variables que estan en el dataset. Para hacer esto, tenemos que plantear un problema de regresion en el que emplearan tecnicas de preprocesamiento como pueden ser la manipulacion de valores faltantes y la eliminacion de las columnas irrelevantes. Ademas se evaluara distintos modelos. Todo esto nos permite identificar la relacion entre las caracteristicas del paciente con su edad.

## 1. Preparacion del Dataset para la Regresion

### 1.1 Seleccionar la Variable Objetivo y las caracteristicas

Como queremos predecir la edad (AGE), la variable objetivo sera "y_age" y las caracteristicas seran el resto de columnas que creamos utiles. 

In [14]:
X_age = df.drop(columns=['AGE'])
y_age = df['AGE']

## 1.2 Division de los datos tanto en el Entrenamiento como en la Prueba

Tenemos que dividir el dataset en "Entrenamiento" y en "Prueba" para poder evaluar la capacidad de generalizacion del modelo.

Lo dividiremos en 80% entrenamiento y 20% prueba

In [15]:
X_train_age, X_test_age, y_train_age, y_test_age = train_test_split(X_age, y_age, test_size=0.2, random_state=42)


## 1.3 Comprobacion de valores faltantes

Antes de entrenar el modelo, tenemos que comprobar los datos faltantes como paso en la seccion O1. Por lo que usaremos "SimpleImputer" para rellenar los valores que falten (NaN). En el caso de las variables numericas podemos usar la "mediana".


In [16]:
imputer_age = SimpleImputer(strategy='median')

X_train_age_imputed = imputer_age.fit_transform(X_train_age)
X_test_age_imputed = imputer_age.transform(X_test_age)

ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: '9999-99-99'

## 1.4 Seleccion y entrenamiento del modelo de regresion

Podemos hacer dos tipos de regresion: Lineal Simple y RandomForestRegressor (No lineal)

### Opcion 1: Regreaion Lineal Simple

En esta opcion, entrenamos una regresion lineal, la cual es eun modelo base para evaluar la relacion lineal entre las caracteristicas utiles y la edad.

In [ ]:
model_lr_age = LinearRegression()
model_lr_age.fit(X_train_age_imputed, y_train_age)

y_pred_lr_age = model_lr_age.predict(X_test_age_imputed)

mse_lr = mean_squared_error(y_test_age, y_pred_lr_age)
r2_lr = r2_score(y_test_age, y_pred_lr_age)
print("MSE (Regresión Lineal):", mse_lr)
print("R2 (Regresión Lineal):", r2_lr)
